# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging that the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what you could sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

In [61]:
%load_ext autoreload
%autoreload 2

In [63]:
# We've scaffolded your classes in the provided lib.py file

from lib import MongoHandler, WeatherGetter
import pandas as pd

In [5]:
getter = WeatherGetter()

In [6]:
getter.token

'01529fc8eb58b47d9bf0422f27e0d8d8'

In [7]:
import sqlite3

conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [10]:
cur.execute('''
SELECT name FROM sqlite_master
WHERE type = 'table' AND name NOT LIKE 'sqlite_%'
;''').fetchall()

[('Matches',), ('Teams_in_Matches',), ('Teams',), ('Unique_Teams',)]

In [12]:
cur.execute('''
PRAGMA table_info(Matches)
;''').fetchall()

[(0, 'Match_ID', 'INTEGER', 1, None, 1),
 (1, 'Div', 'TEXT', 0, None, 0),
 (2, 'Season', 'INT', 0, None, 0),
 (3, 'Date', 'TEXT', 0, None, 0),
 (4, 'HomeTeam', 'TEXT', 0, None, 0),
 (5, 'AwayTeam', 'TEXT', 0, None, 0),
 (6, 'FTHG', 'INT', 0, None, 0),
 (7, 'FTAG', 'INT', 0, None, 0),
 (8, 'FTR', 'TEXT', 0, None, 0)]

In [31]:
cur.execute('''
SELECT * FROM Matches
WHERE (Div = 'D1' OR Div = 'D2')
AND Season = 2011
;''')                 #Select all german games(since we are only looking at weather in Berlin)

german_matches_2011 = pd.DataFrame(cur.fetchall())
german_matches_2011.columns = [x[0] for x in cur.description]
german_matches_2011.head()


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [32]:
german_matches_2011.describe()

,Match_ID,Season,FTHG,FTAG
count,612.000000,612.0,612.000000,612.000000
mean,1397.500000,2011.0,1.627451,1.199346
std,176.813461,0.0,1.355753,1.201254
min,1092.000000,2011.0,0.000000,0.000000
25%,1244.750000,2011.0,1.000000,0.000000
50%,1397.500000,2011.0,1.000000,1.000000
75%,1550.250000,2011.0,2.000000,2.000000
max,1703.000000,2011.0,7.000000,6.000000


In [35]:
cur.execute('''
SELECT DISTINCT(HomeTeam) AS Team
FROM Matches
WHERE (Div = 'D1' OR Div = 'D2')
AND Season = 2011
;''')
german_teams = pd.DataFrame(cur.fetchall())
german_teams.columns = [x[0] for x in cur.description]
german_teams


,Team
0,Nurnberg
1,Stuttgart
2,Wolfsburg
3,Mainz
4,Freiburg
5,M'gladbach
6,Hamburg
7,Werder Bremen
8,Schalke 04
9,Hannover


In [42]:
# Using this to count the goals 
# scored when team played at home

german_teams_home_goals = german_matches_2011.groupby(['HomeTeam']).sum()
german_teams_home_goals.head()

,Match_ID,Season,FTHG,FTAG
HomeTeam,,,,
Aachen,24768,34187,15,24
Augsburg,22397,34187,20,19
Bayern Munich,22898,34187,49,6
Bochum,24898,34187,23,23
Braunschweig,24745,34187,21,15


In [43]:
# select just the FTHG column
german_teams_home_goals = german_teams_home_goals['FTHG']
german_teams_home_goals

HomeTeam
Aachen                15
Augsburg              20
Bayern Munich         49
Bochum                23
Braunschweig          21
Cottbus               18
Dortmund              44
Dresden               30
Duisburg              23
Ein Frankfurt         38
Erzgebirge Aue        19
FC Koln               20
Fortuna Dusseldorf    35
Frankfurt FSV         20
Freiburg              24
Greuther Furth        46
Hamburg               19
Hannover              31
Hansa Rostock         20
Hertha                19
Hoffenheim            21
Ingolstadt            28
Kaiserslautern        12
Karlsruhe             23
Leverkusen            28
M'gladbach            29
Mainz                 27
Munich 1860           35
Nurnberg              22
Paderborn             28
Schalke 04            47
St Pauli              36
Stuttgart             36
Union Berlin          30
Werder Bremen         31
Wolfsburg             29
Name: FTHG, dtype: int64

In [40]:
# Create a similar table to see how many goals were
# scored by each team when playing away

german_teams_away_goals = german_matches_2011.groupby(['AwayTeam']).sum()
german_teams_away_goals.head()

,Match_ID,Season,FTHG,FTAG
AwayTeam,,,,
Aachen,26214,34187,23,15
Augsburg,27659,34187,30,16
Bayern Munich,18700,34187,16,28
Bochum,23035,34187,32,18
Braunschweig,26792,34187,20,16


In [44]:
german_teams_away_goals = german_teams_away_goals['FTAG']
german_teams_away_goals

AwayTeam
Aachen                15
Augsburg              16
Bayern Munich         28
Bochum                18
Braunschweig          16
Cottbus               12
Dortmund              36
Dresden               20
Duisburg              19
Ein Frankfurt         38
Erzgebirge Aue        12
FC Koln               19
Fortuna Dusseldorf    29
Frankfurt FSV         23
Freiburg              21
Greuther Furth        27
Hamburg               16
Hannover              10
Hansa Rostock         14
Hertha                19
Hoffenheim            20
Ingolstadt            15
Kaiserslautern        12
Karlsruhe             11
Leverkusen            24
M'gladbach            20
Mainz                 20
Munich 1860           27
Nurnberg              16
Paderborn             23
Schalke 04            27
St Pauli              23
Stuttgart             27
Union Berlin          25
Werder Bremen         18
Wolfsburg             18
Name: FTAG, dtype: int64

In [52]:
german_teams_df = pd.concat([german_teams_home_goals, german_teams_away_goals], axis = 1)

In [53]:
german_teams_df.head()

,FTHG,FTAG
Aachen,15,15
Augsburg,20,16
Bayern Munich,49,28
Bochum,23,18
Braunschweig,21,16


In [54]:
german_teams_df['Total goals'] = (german_teams_goals['FTHG']
                                    + german_teams_goals['FTAG'])

german_teams_df.head()

,FTHG,FTAG,Total goals
Aachen,15,15,30
Augsburg,20,16,36
Bayern Munich,49,28,77
Bochum,23,18,41
Braunschweig,21,16,37


In [50]:
num_home_games = german_matches_2011.groupby(['HomeTeam']).count()['Match_ID']
num_home_games

HomeTeam
Aachen                17
Augsburg              17
Bayern Munich         17
Bochum                17
Braunschweig          17
Cottbus               17
Dortmund              17
Dresden               17
Duisburg              17
Ein Frankfurt         17
Erzgebirge Aue        17
FC Koln               17
Fortuna Dusseldorf    17
Frankfurt FSV         17
Freiburg              17
Greuther Furth        17
Hamburg               17
Hannover              17
Hansa Rostock         17
Hertha                17
Hoffenheim            17
Ingolstadt            17
Kaiserslautern        17
Karlsruhe             17
Leverkusen            17
M'gladbach            17
Mainz                 17
Munich 1860           17
Nurnberg              17
Paderborn             17
Schalke 04            17
St Pauli              17
Stuttgart             17
Union Berlin          17
Werder Bremen         17
Wolfsburg             17
Name: Match_ID, dtype: int64

In [51]:
num_away_games = german_matches_2011.groupby(['AwayTeam']).count()['Match_ID']
num_away_games

AwayTeam
Aachen                17
Augsburg              17
Bayern Munich         17
Bochum                17
Braunschweig          17
Cottbus               17
Dortmund              17
Dresden               17
Duisburg              17
Ein Frankfurt         17
Erzgebirge Aue        17
FC Koln               17
Fortuna Dusseldorf    17
Frankfurt FSV         17
Freiburg              17
Greuther Furth        17
Hamburg               17
Hannover              17
Hansa Rostock         17
Hertha                17
Hoffenheim            17
Ingolstadt            17
Kaiserslautern        17
Karlsruhe             17
Leverkusen            17
M'gladbach            17
Mainz                 17
Munich 1860           17
Nurnberg              17
Paderborn             17
Schalke 04            17
St Pauli              17
Stuttgart             17
Union Berlin          17
Werder Bremen         17
Wolfsburg             17
Name: Match_ID, dtype: int64

In [55]:
german_teams_df['Matches Played'] = num_home_games + num_away_games

In [56]:
german_teams_df

,FTHG,FTAG,Total goals,Matches Played
Aachen,15,15,30,34
Augsburg,20,16,36,34
Bayern Munich,49,28,77,34
Bochum,23,18,41,34
Braunschweig,21,16,37,34
Cottbus,18,12,30,34
Dortmund,44,36,80,34
Dresden,30,20,50,34
Duisburg,23,19,42,34
Ein Frankfurt,38,38,76,34


In [58]:
# Want to add number of wins for each team

home_wins = german_matches_2011[german_matches_2011['FTR'] == 'H'].groupby(['HomeTeam']).count()['FTR']
home_wins

HomeTeam
Aachen                 4
Augsburg               6
Bayern Munich         14
Bochum                 7
Braunschweig           6
Cottbus                4
Dortmund              14
Dresden                8
Duisburg               8
Ein Frankfurt         11
Erzgebirge Aue         5
FC Koln                5
Fortuna Dusseldorf    11
Frankfurt FSV          3
Freiburg               6
Greuther Furth        14
Hamburg                3
Hannover              10
Hansa Rostock          3
Hertha                 4
Hoffenheim             4
Ingolstadt             6
Kaiserslautern         2
Karlsruhe              8
Leverkusen             8
M'gladbach             9
Mainz                  7
Munich 1860           10
Nurnberg               6
Paderborn              9
Schalke 04            13
St Pauli              12
Stuttgart             10
Union Berlin          11
Werder Bremen          8
Wolfsburg             10
Name: FTR, dtype: int64

In [59]:
away_wins = german_matches_2011[german_matches_2011['FTR'] == 'A'].groupby(['AwayTeam']).count()['FTR']
away_wins

AwayTeam
Aachen                 2
Augsburg               2
Bayern Munich          9
Bochum                 3
Braunschweig           4
Cottbus                4
Dortmund              11
Dresden                4
Duisburg               2
Ein Frankfurt          9
Erzgebirge Aue         3
FC Koln                3
Fortuna Dusseldorf     5
Frankfurt FSV          4
Freiburg               4
Greuther Furth         6
Hamburg                5
Hannover               2
Hansa Rostock          2
Hertha                 3
Hoffenheim             6
Ingolstadt             2
Kaiserslautern         2
Karlsruhe              1
Leverkusen             7
M'gladbach             8
Mainz                  2
Munich 1860            7
Nurnberg               6
Paderborn              8
Schalke 04             7
St Pauli               6
Stuttgart              5
Union Berlin           3
Werder Bremen          3
Wolfsburg              3
Name: FTR, dtype: int64

In [60]:
german_teams_df['Num wins'] = home_wins + away_wins
german_teams_df.head()

,FTHG,FTAG,Total goals,Matches Played,Num wins
Aachen,15,15,30,34,6
Augsburg,20,16,36,34,8
Bayern Munich,49,28,77,34,23
Bochum,23,18,41,34,10
Braunschweig,21,16,37,34,10


At this point I want to add the weather data, count how may matches were played in the rain and subsequently, how many rainy games played were won by each team.

In [77]:
# test the weather getter class
# berlin coordinates from google
berlin_lat = '52.5200'
berlin_lon = '13.4050'
getter = WeatherGetter()
data = getter.get_weather(berlin_lat, berlin_lon, '2012-03-31') # using the first date given in the german_matches_2011 df

In [78]:
data.keys()

dict_keys(['latitude', 'longitude', 'timezone', 'currently', 'offset'])

In [79]:
data['currently']

{'time': 1333198800,
 'summary': 'Mostly Cloudy',
 'icon': 'partly-cloudy-day',
 'precipIntensity': 0,
 'precipProbability': 0,
 'temperature': 5.17,
 'apparentTemperature': 1.25,
 'dewPoint': 1.29,
 'humidity': 0.76,
 'windSpeed': 5.66,
 'windGust': 11.31,
 'windBearing': 311,
 'cloudCover': 0.75,
 'uvIndex': 3,
 'visibility': 10.003}

In [81]:
data['currently']['precipIntensity']

0

In [83]:
for date in german_matches_2011['Date'].iloc[:5]:
    print (date)

2012-03-31
2011-12-11
2011-08-13
2011-11-27
2012-02-18


In [91]:
dates = []
for date in german_matches_2011['Date'].iloc[:5]:
    dates.append(date)

weather_dict = getter.get_weather_dates(berlin_lat, berlin_lon, dates)
weather_dict

{'2012-03-31': {'time': 1333198800,
  'summary': 'Mostly Cloudy',
  'icon': 'partly-cloudy-day',
  'precipIntensity': 0,
  'precipProbability': 0,
  'temperature': 5.17,
  'apparentTemperature': 1.25,
  'dewPoint': 1.29,
  'humidity': 0.76,
  'windSpeed': 5.66,
  'windGust': 11.31,
  'windBearing': 311,
  'cloudCover': 0.75,
  'uvIndex': 3,
  'visibility': 10.003},
 '2011-12-11': {'time': 1323612000,
  'summary': 'Mostly Cloudy',
  'icon': 'partly-cloudy-day',
  'precipIntensity': 0,
  'precipProbability': 0,
  'temperature': 3.69,
  'apparentTemperature': 0.52,
  'dewPoint': -1.45,
  'humidity': 0.69,
  'windSpeed': 3.57,
  'windGust': 3.57,
  'windBearing': 191,
  'cloudCover': 0.75,
  'uvIndex': 0,
  'visibility': 9.996},
 '2011-08-13': {'time': 1313240400,
  'summary': 'Partly Cloudy',
  'icon': 'partly-cloudy-day',
  'precipIntensity': 0,
  'precipProbability': 0,
  'temperature': 22.19,
  'apparentTemperature': 22.19,
  'dewPoint': 12.36,
  'humidity': 0.54,
  'windSpeed': 4.12,


I have examined the data that I will receive from the DarkSky API. My weather getter class now has methods that will create a weather data dictionary. I will feed it the dates from my matches df to obtain all the data I need. I will examine all the data to make sure it is clean. After that, I will update my dataframe with rain data.

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!